In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
# IDP_df = pd.read_pickle('male_IDP.pkl')
vars_df = pd.read_pickle('female_vars.pkl')
# IDP_names = np.loadtxt("IDP_names.txt", dtype=str, delimiter='\n')
# IDP_categories = np.loadtxt("IDP_categories.txt", dtype=str, delimiter='\n')
with open('varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
vars_categories = np.loadtxt("vars_categories.txt", dtype=str, delimiter='\n')

In [3]:
vars_df.head()

,modality,bonf,fdr,dataframe
0,T1_nonlinear,5.441192,3.341511,idx ...
1,T1_linear,5.441192,3.403566,idx ...
2,jacobian,5.441192,3.392838,idx ...
3,vbm,5.441192,3.441654,idx ...
4,T2_nonlinear,5.441192,3.654097,idx ...


In [4]:
vars_df.iloc[0].dataframe.head()

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
0,0,Ethnic background (0.0),Ethnic Background,0.001153,0.081615,1.000000,0.934956,0.001153,0.029209,4.821637e-17
1,1,Ethnic background (1.0),Ethnic Background,-0.002082,0.060202,1.000000,0.952009,0.002082,0.021359,4.821637e-17
2,2,Ethnic background (2.0),Ethnic Background,-0.015050,0.567786,1.000000,0.570270,0.015050,0.243919,4.821637e-17
3,3,Genotype measurement batch (0.0),Genetic Markers,-0.026108,1.824075,0.665558,0.068202,0.026108,1.166203,1.768139e-01
4,4,Heterozygosity (0.0),Genetic Markers,-0.035325,2.468699,0.401285,0.013595,0.035325,1.866635,3.965472e-01


In [5]:
vars_over_bThr_list = []

for idx in range(len(vars_df)):
    df = vars_df.iloc[idx].dataframe
    bThr = vars_df.iloc[idx].bonf
    vars_over_bThr = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False).names.to_list()
    for var in vars_over_bThr:
        vars_over_bThr_list.append(var)

In [6]:
print("Number of unique vars across all modalities: ", len(set(vars_over_bThr_list)))

Number of unique vars across all modalities:  214


In [7]:
unique_vars_over_bTHr = list(set(vars_over_bThr_list))

In [8]:
unique_vars_over_bThr_categories = []

for var in unique_vars_over_bTHr:
    idx = list(varsHeader).index(var)
    category = vars_categories[idx]
    unique_vars_over_bThr_categories.append(category)

In [9]:
sorted(unique_vars_over_bTHr)

['Alcohol (2.0)',
 'Alcohol intake frequency. (0.0)',
 'Alcohol intake frequency. (2.0)',
 'Amount of alcohol drunk on a typical drinking day (0.0)',
 'Android bone mass (2.0)',
 'Android tissue fat percentage (2.0)',
 'Android total mass (2.0)',
 'Arm BMD (bone mineral density) (left) (2.0)',
 'Arm BMD (bone mineral density) (right) (2.0)',
 'Arm fat mass (right) (0.0)',
 'Arm fat mass (right) (2.0)',
 'Arm fat percentage (left) (0.0)',
 'Arm fat percentage (left) (2.0)',
 'Arm fat percentage (right) (0.0)',
 'Arm fat percentage (right) (2.0)',
 'Arm fat-free mass (left) (0.0)',
 'Arm fat-free mass (right) (0.0)',
 'Arm fat-free mass (right) (2.0)',
 'Arm predicted mass (left) (0.0)',
 'Arm tissue fat percentage (left) (2.0)',
 'Arm total mass (left) (2.0)',
 'Arm total mass (right) (2.0)',
 'Arms BMC (bone mineral content) (2.0)',
 'Arms BMD (bone mineral density) (2.0)',
 'Arms tissue fat percentage (2.0)',
 'Arms total mass (2.0)',
 'Average weekly red wine intake (0.0)',
 'Average

In [10]:
modalities = []
pearson_rs_list = []
for var in unique_vars_over_bTHr:
    modalities_with_var = []
    pearsonRs_with_var = []
    for idx in range(len(vars_df)):
        df = vars_df.iloc[idx].dataframe
        modality = vars_df.iloc[idx].modality
        bThr = vars_df.iloc[idx].bonf
        df = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False)
        vars_over_bThr = df.names.to_list()
        pearson_r_over_bThr = df.pearson_r.to_list()
        if var in vars_over_bThr:
            modalities_with_var.append(modality)
            pearsonRs_with_var.append(pearson_r_over_bThr[vars_over_bThr.index(var)])
    modalities.append(modalities_with_var)
    pearson_rs_list.append(pearsonRs_with_var)

In [11]:
len(modalities)

214

In [12]:
df_vars_over_bThr = pd.DataFrame.from_dict({
    'names': unique_vars_over_bTHr,
    'Categories': unique_vars_over_bThr_categories,
    'modalities': modalities,
    'pearson_r': pearson_rs_list
})

In [13]:
df_vars_over_bThr.head()

,names,Categories,modalities,pearson_r
0,Diagnoses - ICD10 (I10 - I10 Essential (primar...,Medical History,"[T1_nonlinear, T1_linear, T2_lesions, tbss_FA_...","[0.07194972143697789, 0.06784801360709078, 0.1..."
1,Diabetes diagnosed by doctor (2.0),Medical History,"[T2_nonlinear, tbss_ICVF]","[0.06692855559504764, 0.06637856552501097]"
2,Total BMD (bone mineral density) (2.0),Skeletal Measurements,"[T1_nonlinear, T1_linear, jacobian, vbm, T2_no...","[-0.11110788441466599, -0.15040438165962552, -..."
3,Number of treatments/medications taken (2.0),Medical History,"[T2_lesions, tracts, tbss_FA_s, tbss_ISOVF_s, ...","[0.08158918954911325, 0.0715406561162527, 0.08..."
4,Standing height (2.0),Physical Measurements,[rsfmri_2],[-0.06737539994654279]


In [14]:
df_vars_over_bThr[df_vars_over_bThr.names=='Arm fat mass (right) (0.0)']

,names,Categories,modalities,pearson_r
151,Arm fat mass (right) (0.0),Physical Measurements,"[rsfmri_0, rsfmri_1, rsfmri_2, rsfmri_4, rsfmr...","[-0.08720660964021588, -0.06884466507645581, -..."


In [15]:
set(df_vars_over_bThr.Categories.to_list())

{'Alcohol',
 'Blood Assays',
 'Cardiac & Circulartory Measurements',
 'Cognitive Tests',
 'Diet',
 'Medical History',
 'Physical Measurements',
 'Skeletal Measurements',
 'Tobacco'}

In [16]:
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        print('   Modalities:')
        for i, mod in enumerate(mods[0]):
            print('   -> {} ({:.3f})'.format(mod, prs[0][i]))
        
    print('\n')

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> jacobian (0.078)
   -> vbm (0.074)
   -> rsfmri_0 (0.072)
   -> rsfmri_4 (0.066)
   -> tfmri_1 (0.066)
   -> tfmri_c_2 (0.066)
   -> tbss_FA_s (0.076)
   -> tbss_ISOVF_s (0.095)
   -> tbss_L1_s (0.098)
   -> tbss_L2_s (0.094)
   -> tbss_L3_s (0.092)
   -> tbss_MD_s (0.088)
   -> tbss_OD_s (0.072)
   -> tbss_FA (0.089)
   -> tbss_ICVF (0.066)
   -> tbss_ISOVF (0.084)
   -> tbss_L1 (0.084)
   -> tbss_L2 (0.083)
   -> tbss_L3 (0.078)
   -> tbss_MD (0.092)
   -> tbss_OD (0.084)
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> jacobian (0.079)
   -> vbm (0.077)
   -> swi (0.068)
   -> rsfmri_0 (0.082)
   -> rsfmri_7 (0.075)
   -> rsfmri_18 (0.069)
   -> rsfmri_19 (0.069)
   -> rsfmri_23 (0.069)
   -> tfmri_1 (0.074)
   -> tfmri_c_2 (0.067)
   -> tracts (0.066)
   -> tbss_FA_s (0.085)
   -> tbss_ICVF_s (0.066)
   -> tbss_ISOVF_s (0.097)
   -> tbss_L1_s (0.086)

   Modalities:
   -> rsfmri_18 (0.070)
   -> rsfmri_23 (0.070)
   -> tfmri_2 (0.071)
   -> tfmri_c_1 (0.079)
   -> tfmri_c_2 (0.066)
Var: Treatment/medication code (1140883066 - insulin product)
   Modalities:
   -> tbss_ISOVF_s (0.073)
   -> tbss_L1_s (0.067)
   -> tbss_OD_s (0.079)
   -> tbss_ICVF (0.067)
   -> tbss_ISOVF (0.072)
   -> tbss_L1 (0.077)
   -> tbss_L2 (0.077)
   -> tbss_L3 (0.066)
   -> tbss_MD (0.066)
   -> tbss_OD (0.065)
Var: Treatment/medication code (1140884600 - metformin)
   Modalities:
   -> T2_nonlinear (0.067)
   -> tracts (0.066)
   -> tbss_ISOVF_s (0.067)
Var: Vascular/heart problems diagnosed by doctor (-7.0 - None of the above)
   Modalities:
   -> T2_lesions (-0.082)
   -> tbss_FA_s (-0.077)
   -> tbss_ISOVF_s (-0.077)
   -> tbss_L2_s (-0.070)
   -> tbss_L3_s (-0.076)
   -> tbss_MD_s (-0.068)
   -> tbss_FA (-0.066)
   -> tbss_L2 (-0.068)
   -> tbss_L3 (-0.071)
Var: Vascular/heart problems diagnosed by doctor (4 - High blood pressure)
   Modalities:
   -> 

   Modalities:
   -> jacobian (-0.073)
   -> swi (-0.075)
   -> rsfmri_0 (-0.081)
   -> rsfmri_1 (-0.085)
   -> rsfmri_2 (-0.099)
   -> rsfmri_3 (-0.092)
   -> rsfmri_4 (-0.094)
   -> rsfmri_5 (-0.081)
   -> rsfmri_6 (-0.096)
   -> rsfmri_7 (-0.094)
   -> rsfmri_8 (-0.089)
   -> rsfmri_9 (-0.074)
   -> rsfmri_10 (-0.089)
   -> rsfmri_11 (-0.119)
   -> rsfmri_12 (-0.067)
   -> rsfmri_13 (-0.086)
   -> rsfmri_14 (-0.095)
   -> rsfmri_15 (-0.089)
   -> rsfmri_17 (-0.107)
   -> rsfmri_18 (-0.071)
   -> rsfmri_19 (-0.087)
   -> rsfmri_20 (-0.081)
   -> rsfmri_21 (-0.102)
   -> rsfmri_23 (-0.080)
   -> rsfmri_24 (-0.084)
   -> tfmri_2 (-0.075)
   -> tfmri_5 (-0.079)
   -> tfmri_c_2 (-0.081)
   -> tfmri_c_5 (-0.087)
   -> tracts (-0.076)
   -> tbss_ISOVF_s (-0.098)
   -> tbss_L1_s (-0.087)
   -> tbss_L2_s (-0.066)
   -> tbss_L3_s (-0.068)
   -> tbss_MO_s (-0.076)
   -> tbss_OD_s (-0.080)
   -> tbss_FA (-0.088)
   -> tbss_ISOVF (-0.079)
   -> tbss_L1 (-0.087)
   -> tbss_L2 (-0.067)
   -> tbss_

   Modalities:
   -> swi (-0.069)
   -> rsfmri_0 (-0.069)
   -> rsfmri_2 (-0.097)
   -> rsfmri_3 (-0.083)
   -> rsfmri_5 (-0.086)
   -> rsfmri_6 (-0.077)
   -> rsfmri_7 (-0.071)
   -> rsfmri_9 (-0.069)
   -> rsfmri_10 (-0.076)
   -> rsfmri_11 (-0.096)
   -> rsfmri_12 (-0.079)
   -> rsfmri_13 (-0.081)
   -> rsfmri_14 (-0.087)
   -> rsfmri_16 (-0.080)
   -> rsfmri_17 (-0.085)
   -> rsfmri_18 (-0.075)
   -> rsfmri_19 (-0.077)
   -> rsfmri_20 (-0.081)
   -> rsfmri_21 (-0.100)
   -> rsfmri_23 (-0.070)
   -> tfmri_2 (-0.069)
   -> tfmri_5 (-0.082)
   -> tfmri_c_2 (-0.081)
   -> tfmri_c_5 (-0.083)
Var: Leg total mass (left) (2.0)
   Modalities:
   -> rsfmri_11 (-0.116)
   -> tbss_OD (-0.116)
Var: Leg total mass (right) (2.0)
   Modalities:
   -> rsfmri_11 (-0.123)
Var: Legs total mass (2.0)
   Modalities:
   -> jacobian (-0.074)
   -> swi (-0.078)
   -> rsfmri_0 (-0.103)
   -> rsfmri_1 (-0.095)
   -> rsfmri_2 (-0.120)
   -> rsfmri_3 (-0.095)
   -> rsfmri_4 (-0.097)
   -> rsfmri_5 (-0.107)
   

   Modalities:
   -> T1_nonlinear (-0.066)
   -> jacobian (-0.076)
   -> swi (-0.082)
   -> rsfmri_0 (-0.099)
   -> rsfmri_1 (-0.090)
   -> rsfmri_2 (-0.125)
   -> rsfmri_3 (-0.105)
   -> rsfmri_4 (-0.095)
   -> rsfmri_5 (-0.105)
   -> rsfmri_6 (-0.102)
   -> rsfmri_7 (-0.101)
   -> rsfmri_8 (-0.087)
   -> rsfmri_9 (-0.098)
   -> rsfmri_10 (-0.108)
   -> rsfmri_11 (-0.137)
   -> rsfmri_12 (-0.088)
   -> rsfmri_13 (-0.104)
   -> rsfmri_14 (-0.105)
   -> rsfmri_15 (-0.093)
   -> rsfmri_16 (-0.093)
   -> rsfmri_17 (-0.126)
   -> rsfmri_18 (-0.098)
   -> rsfmri_19 (-0.092)
   -> rsfmri_20 (-0.106)
   -> rsfmri_21 (-0.125)
   -> rsfmri_22 (-0.073)
   -> rsfmri_23 (-0.100)
   -> rsfmri_24 (-0.087)
   -> tfmri_1 (-0.080)
   -> tfmri_2 (-0.096)
   -> tfmri_5 (-0.094)
   -> tfmri_c_1 (-0.088)
   -> tfmri_c_2 (-0.104)
   -> tfmri_c_5 (-0.095)
   -> tracts (-0.077)
   -> tbss_FA_s (-0.069)
   -> tbss_ISOVF_s (-0.093)
   -> tbss_L1_s (-0.093)
   -> tbss_L2_s (-0.068)
   -> tbss_MO_s (-0.095)
   ->

   -> tbss_L3 (-0.131)
   -> tbss_MD (-0.141)
   -> tbss_MO (-0.141)
   -> tbss_OD (-0.135)
Var: Femur neck BMD (bone mineral density) (right) (2.0)
   Modalities:
   -> T1_nonlinear (-0.095)
   -> T1_linear (-0.124)
   -> jacobian (-0.178)
   -> vbm (-0.133)
   -> T2_nonlinear (-0.071)
   -> T2_lesions (-0.078)
   -> swi (-0.164)
   -> rsfmri_0 (-0.230)
   -> rsfmri_1 (-0.191)
   -> rsfmri_2 (-0.244)
   -> rsfmri_3 (-0.154)
   -> rsfmri_4 (-0.204)
   -> rsfmri_5 (-0.242)
   -> rsfmri_6 (-0.221)
   -> rsfmri_7 (-0.223)
   -> rsfmri_8 (-0.205)
   -> rsfmri_9 (-0.232)
   -> rsfmri_10 (-0.218)
   -> rsfmri_11 (-0.234)
   -> rsfmri_12 (-0.200)
   -> rsfmri_13 (-0.233)
   -> rsfmri_14 (-0.176)
   -> rsfmri_15 (-0.209)
   -> rsfmri_16 (-0.203)
   -> rsfmri_17 (-0.225)
   -> rsfmri_18 (-0.184)
   -> rsfmri_19 (-0.151)
   -> rsfmri_20 (-0.213)
   -> rsfmri_21 (-0.239)
   -> rsfmri_22 (-0.173)
   -> rsfmri_23 (-0.174)
   -> rsfmri_24 (-0.191)
   -> tfmri_1 (-0.161)
   -> tfmri_2 (-0.179)
   -> 

   -> rsfmri_21 (-0.225)
   -> rsfmri_22 (-0.158)
   -> rsfmri_23 (-0.166)
   -> rsfmri_24 (-0.191)
   -> tfmri_1 (-0.153)
   -> tfmri_2 (-0.159)
   -> tfmri_5 (-0.189)
   -> tfmri_c_1 (-0.141)
   -> tfmri_c_2 (-0.155)
   -> tfmri_c_5 (-0.159)
   -> tracts (-0.106)
   -> tbss_FA_s (-0.078)
   -> tbss_ICVF_s (-0.093)
   -> tbss_ISOVF_s (-0.131)
   -> tbss_L1_s (-0.123)
   -> tbss_L2_s (-0.106)
   -> tbss_L3_s (-0.107)
   -> tbss_MD_s (-0.102)
   -> tbss_MO_s (-0.090)
   -> tbss_OD_s (-0.104)
   -> tbss_FA (-0.136)
   -> tbss_ICVF (-0.114)
   -> tbss_ISOVF (-0.130)
   -> tbss_L1 (-0.138)
   -> tbss_L2 (-0.112)
   -> tbss_L3 (-0.114)
   -> tbss_MD (-0.132)
   -> tbss_MO (-0.138)
   -> tbss_OD (-0.125)
Var: Femur troch BMD (bone mineral density) T-score (left) (2.0)
   Modalities:
   -> T1_nonlinear (-0.095)
   -> T1_linear (-0.119)
   -> jacobian (-0.173)
   -> vbm (-0.117)
   -> swi (-0.170)
   -> rsfmri_0 (-0.219)
   -> rsfmri_1 (-0.197)
   -> rsfmri_2 (-0.220)
   -> rsfmri_3 (-0.170)
 

   -> tfmri_c_5 (-0.191)
   -> tracts (-0.137)
   -> tbss_FA_s (-0.152)
   -> tbss_ICVF_s (-0.134)
   -> tbss_ISOVF_s (-0.182)
   -> tbss_L1_s (-0.169)
   -> tbss_L2_s (-0.153)
   -> tbss_L3_s (-0.158)
   -> tbss_MD_s (-0.136)
   -> tbss_MO_s (-0.165)
   -> tbss_OD_s (-0.132)
   -> tbss_FA (-0.175)
   -> tbss_ICVF (-0.127)
   -> tbss_ISOVF (-0.139)
   -> tbss_L1 (-0.158)
   -> tbss_L2 (-0.132)
   -> tbss_L3 (-0.141)
   -> tbss_MD (-0.163)
   -> tbss_MO (-0.181)
   -> tbss_OD (-0.133)
Var: Head BMD (bone mineral density) (2.0)
   Modalities:
   -> T1_nonlinear (-0.077)
   -> T1_linear (-0.110)
   -> jacobian (-0.212)
   -> vbm (-0.183)
   -> T2_lesions (-0.073)
   -> swi (-0.192)
   -> rsfmri_0 (-0.323)
   -> rsfmri_1 (-0.315)
   -> rsfmri_2 (-0.349)
   -> rsfmri_3 (-0.252)
   -> rsfmri_4 (-0.314)
   -> rsfmri_5 (-0.328)
   -> rsfmri_6 (-0.327)
   -> rsfmri_7 (-0.312)
   -> rsfmri_8 (-0.307)
   -> rsfmri_9 (-0.343)
   -> rsfmri_10 (-0.305)
   -> rsfmri_11 (-0.324)
   -> rsfmri_12 (-0.29

   -> jacobian (-0.147)
   -> vbm (-0.092)
   -> swi (-0.112)
   -> rsfmri_0 (-0.212)
   -> rsfmri_1 (-0.165)
   -> rsfmri_2 (-0.170)
   -> rsfmri_3 (-0.140)
   -> rsfmri_4 (-0.177)
   -> rsfmri_5 (-0.185)
   -> rsfmri_6 (-0.166)
   -> rsfmri_7 (-0.178)
   -> rsfmri_8 (-0.172)
   -> rsfmri_9 (-0.173)
   -> rsfmri_10 (-0.155)
   -> rsfmri_11 (-0.177)
   -> rsfmri_12 (-0.168)
   -> rsfmri_13 (-0.172)
   -> rsfmri_14 (-0.165)
   -> rsfmri_15 (-0.171)
   -> rsfmri_16 (-0.169)
   -> rsfmri_17 (-0.174)
   -> rsfmri_18 (-0.163)
   -> rsfmri_19 (-0.163)
   -> rsfmri_20 (-0.170)
   -> rsfmri_21 (-0.192)
   -> rsfmri_22 (-0.164)
   -> rsfmri_23 (-0.153)
   -> rsfmri_24 (-0.181)
   -> tfmri_1 (-0.133)
   -> tfmri_2 (-0.137)
   -> tfmri_5 (-0.157)
   -> tfmri_c_1 (-0.126)
   -> tfmri_c_2 (-0.133)
   -> tfmri_c_5 (-0.135)
   -> tracts (-0.094)
   -> tbss_FA_s (-0.081)
   -> tbss_ISOVF_s (-0.100)
   -> tbss_L1_s (-0.103)
   -> tbss_L2_s (-0.094)
   -> tbss_L3_s (-0.116)
   -> tbss_MD_s (-0.094)
   -

   Modalities:
   -> T1_linear (-0.096)
   -> jacobian (-0.140)
   -> vbm (-0.098)
   -> swi (-0.154)
   -> rsfmri_0 (-0.188)
   -> rsfmri_1 (-0.176)
   -> rsfmri_2 (-0.201)
   -> rsfmri_3 (-0.159)
   -> rsfmri_4 (-0.173)
   -> rsfmri_5 (-0.179)
   -> rsfmri_6 (-0.188)
   -> rsfmri_7 (-0.179)
   -> rsfmri_8 (-0.176)
   -> rsfmri_9 (-0.181)
   -> rsfmri_10 (-0.161)
   -> rsfmri_11 (-0.200)
   -> rsfmri_12 (-0.167)
   -> rsfmri_13 (-0.186)
   -> rsfmri_14 (-0.169)
   -> rsfmri_15 (-0.179)
   -> rsfmri_16 (-0.185)
   -> rsfmri_17 (-0.194)
   -> rsfmri_18 (-0.167)
   -> rsfmri_19 (-0.149)
   -> rsfmri_20 (-0.157)
   -> rsfmri_21 (-0.204)
   -> rsfmri_22 (-0.161)
   -> rsfmri_23 (-0.171)
   -> rsfmri_24 (-0.155)
   -> tfmri_1 (-0.126)
   -> tfmri_2 (-0.147)
   -> tfmri_5 (-0.174)
   -> tfmri_c_1 (-0.137)
   -> tfmri_c_2 (-0.144)
   -> tfmri_c_5 (-0.154)
   -> tracts (-0.101)
   -> tbss_FA_s (-0.096)
   -> tbss_ICVF_s (-0.095)
   -> tbss_ISOVF_s (-0.139)
   -> tbss_L1_s (-0.123)
   -> tbss_L

In [17]:
sorted(prs[0], reverse=True)

[0.07142636835075232,
 0.06872484097489105,
 0.06805928920789929,
 0.06646766667482733]

In [18]:
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f})'.format(mod, prs[i]))
        
    print('\n')

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> tbss_L1_s (0.098)
   -> tbss_ISOVF_s (0.095)
   -> tbss_L2_s (0.094)
   -> tbss_MD (0.092)
   -> tbss_L3_s (0.092)
   -> tbss_FA (0.089)
   -> tbss_MD_s (0.088)
   -> tbss_ISOVF (0.084)
   -> tbss_OD (0.084)
   -> tbss_L1 (0.084)
   -> tbss_L2 (0.083)
   -> tbss_L3 (0.078)
   -> jacobian (0.078)
   -> tbss_FA_s (0.076)
   -> vbm (0.074)
   -> rsfmri_0 (0.072)
   -> tbss_OD_s (0.072)
   -> tfmri_c_2 (0.066)
   -> tfmri_1 (0.066)
   -> rsfmri_4 (0.066)
   -> tbss_ICVF (0.066)
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> tbss_ISOVF_s (0.097)
   -> tbss_FA (0.093)
   -> tbss_L2_s (0.091)
   -> tbss_MD (0.090)
   -> tbss_OD (0.087)
   -> tbss_L3_s (0.087)
   -> tbss_L1_s (0.086)
   -> tbss_FA_s (0.085)
   -> tbss_ISOVF (0.083)
   -> rsfmri_0 (0.082)
   -> tbss_OD_s (0.082)
   -> tbss_L2 (0.081)
   -> tbss_MD_s (0.080)
   -> jacobian (0.079)
   -> tbss_ICV

   -> tbss_MD_s (0.085)
   -> rsfmri_3 (0.084)
   -> tbss_FA (0.081)
   -> tbss_FA_s (0.077)
   -> tbss_MO_s (0.076)
   -> tbss_OD_s (0.076)
   -> tbss_OD (0.076)
   -> tbss_L3 (0.074)
   -> tracts (0.070)


Category: Cognitive Tests
------------------------------------------------
Var: Interval between previous point and current one in alphanumeric path (trail #2) (2.12)
   Modalities:
   -> tfmri_c_1 (0.083)


Category: Diet
------------------------------------------------
Var: Alcohol (2.0)
   Modalities:
   -> tbss_ISOVF_s (0.131)
Var: Cereal intake (2.0)
   Modalities:
   -> tbss_ICVF_s (-0.072)
Var: Coffee type (0.0)
   Modalities:
   -> rsfmri_11 (0.076)
Var: Tea intake (0.0)
   Modalities:
   -> rsfmri_5 (-0.069)
   -> rsfmri_10 (-0.067)


Category: Medical History
------------------------------------------------
Var: Diabetes diagnosed by doctor (0.0)
   Modalities:
   -> tbss_ICVF (0.087)
   -> tbss_ISOVF_s (0.085)
   -> tbss_L2 (0.083)
   -> tbss_ICVF_s (0.082)
   -> tbss_IS

   -> rsfmri_11 (-0.116)
   -> rsfmri_17 (-0.102)
   -> rsfmri_4 (-0.100)
   -> rsfmri_21 (-0.100)
   -> tbss_OD (-0.097)
   -> tbss_ISOVF_s (-0.096)
   -> rsfmri_2 (-0.096)
   -> rsfmri_6 (-0.094)
   -> tbss_L1_s (-0.090)
   -> rsfmri_7 (-0.090)
   -> tbss_FA (-0.090)
   -> rsfmri_13 (-0.087)
   -> rsfmri_10 (-0.087)
   -> tfmri_c_5 (-0.085)
   -> rsfmri_3 (-0.081)
   -> rsfmri_1 (-0.079)
   -> rsfmri_9 (-0.079)
   -> rsfmri_15 (-0.078)
   -> tfmri_5 (-0.078)
   -> tbss_OD_s (-0.078)
   -> rsfmri_5 (-0.077)
   -> tfmri_c_2 (-0.077)
   -> rsfmri_20 (-0.076)
   -> rsfmri_14 (-0.075)
   -> tbss_L1 (-0.075)
   -> tbss_L3_s (-0.074)
   -> tbss_L2_s (-0.074)
   -> rsfmri_8 (-0.073)
   -> rsfmri_24 (-0.073)
   -> rsfmri_0 (-0.073)
   -> rsfmri_23 (-0.072)
   -> tracts (-0.072)
   -> jacobian (-0.072)
   -> rsfmri_19 (-0.071)
   -> tfmri_1 (-0.071)
   -> tfmri_2 (-0.070)
   -> tbss_MO_s (-0.070)
   -> tbss_ISOVF (-0.069)
   -> swi (-0.069)
Var: Arm fat percentage (right) (0.0)
   Modalities:


   -> tfmri_c_5 (-0.082)
   -> tfmri_2 (-0.082)
   -> rsfmri_4 (-0.079)
   -> tbss_OD_s (-0.078)
   -> tbss_L1_s (-0.078)
   -> rsfmri_18 (-0.077)
   -> tfmri_1 (-0.076)
   -> tbss_MO (-0.075)
   -> tfmri_c_1 (-0.074)
   -> rsfmri_1 (-0.074)
   -> tbss_MO_s (-0.074)
   -> rsfmri_16 (-0.074)
   -> rsfmri_0 (-0.073)
   -> tbss_ISOVF_s (-0.073)
   -> rsfmri_15 (-0.072)
   -> rsfmri_19 (-0.072)
   -> rsfmri_8 (-0.072)
   -> rsfmri_9 (-0.072)
   -> tbss_L1 (-0.070)
   -> swi (-0.069)
   -> rsfmri_12 (-0.069)
   -> rsfmri_5 (-0.069)
   -> rsfmri_24 (-0.067)
Var: Leg fat mass (right) (0.0)
   Modalities:
   -> rsfmri_2 (-0.110)
   -> rsfmri_11 (-0.107)
   -> rsfmri_21 (-0.104)
   -> rsfmri_20 (-0.094)
   -> rsfmri_7 (-0.093)
   -> rsfmri_0 (-0.092)
   -> rsfmri_17 (-0.088)
   -> rsfmri_13 (-0.087)
   -> rsfmri_14 (-0.086)
   -> tbss_OD_s (-0.085)
   -> rsfmri_9 (-0.083)
   -> rsfmri_10 (-0.082)
   -> rsfmri_5 (-0.081)
   -> tbss_L1_s (-0.080)
   -> rsfmri_4 (-0.079)
   -> tbss_OD (-0.079)
   

   -> rsfmri_14 (-0.079)
   -> rsfmri_0 (-0.079)
   -> rsfmri_1 (-0.079)
   -> tfmri_c_5 (-0.074)
   -> rsfmri_5 (-0.074)
   -> tfmri_1 (-0.073)
   -> tbss_L2_s (-0.072)
   -> tfmri_5 (-0.072)
   -> rsfmri_3 (-0.072)
   -> tfmri_c_2 (-0.072)
   -> tbss_L3_s (-0.071)
   -> rsfmri_18 (-0.071)
   -> tfmri_2 (-0.071)
   -> tbss_MO (-0.070)
   -> tracts (-0.070)
   -> tfmri_c_1 (-0.070)
   -> tbss_MD_s (-0.068)
   -> tbss_FA_s (-0.068)
Var: Trunk total mass (2.0)
   Modalities:
   -> rsfmri_11 (-0.137)
   -> rsfmri_2 (-0.125)
   -> rsfmri_21 (-0.118)
   -> rsfmri_17 (-0.117)
   -> rsfmri_20 (-0.101)
   -> rsfmri_3 (-0.101)
   -> rsfmri_7 (-0.101)
   -> rsfmri_6 (-0.100)
   -> rsfmri_9 (-0.100)
   -> rsfmri_18 (-0.100)
   -> rsfmri_5 (-0.097)
   -> tfmri_c_2 (-0.094)
   -> rsfmri_13 (-0.094)
   -> rsfmri_10 (-0.094)
   -> tfmri_2 (-0.093)
   -> rsfmri_23 (-0.092)
   -> rsfmri_16 (-0.092)
   -> rsfmri_0 (-0.091)
   -> rsfmri_4 (-0.090)
   -> rsfmri_14 (-0.089)
   -> tbss_ISOVF_s (-0.089)
   -

...

   -> tbss_OD (-0.126)
   -> T1_linear (-0.123)
   -> tbss_L1_s (-0.122)
   -> tbss_L3 (-0.115)
   -> tbss_ICVF (-0.113)
   -> tbss_L2 (-0.112)
   -> tracts (-0.106)
   -> tbss_L3_s (-0.106)
   -> tbss_L2_s (-0.105)
   -> tbss_OD_s (-0.102)
   -> tbss_MD_s (-0.101)
   -> T1_nonlinear (-0.099)
   -> tbss_ICVF_s (-0.093)
   -> tbss_MO_s (-0.090)
   -> tbss_FA_s (-0.076)
   -> T2_nonlinear (-0.070)
Var: Femur upper neck BMD (bone mineral density) (left) (2.0)
   Modalities:
   -> rsfmri_21 (-0.246)
   -> rsfmri_6 (-0.243)
   -> rsfmri_9 (-0.240)
   -> rsfmri_5 (-0.231)
   -> rsfmri_11 (-0.229)
   -> rsfmri_0 (-0.225)
   -> rsfmri_2 (-0.219)
   -> rsfmri_1 (-0.219)
   -> rsfmri_10 (-0.213)
   -> rsfmri_15 (-0.209)
   -> rsfmri_12 (-0.208)
   -> rsfmri_14 (-0.207)
   -> rsfmri_13 (-0.199)
   -> tfmri_2 (-0.197)
   -> rsfmri_8 (-0.196)
   -> rsfmri_17 (-0.195)
   -> rsfmri_20 (-0.194)
   -> rsfmri_3 (-0.192)
   -> rsfmri_4 (-0.190)
   -> rsfmri_23 (-0.187)
   -> tfmri_5 (-0.176)
   -> rsfmri

   -> tbss_MO (-0.181)
   -> vbm (-0.177)
   -> tbss_FA (-0.175)
   -> tfmri_c_2 (-0.174)
   -> tfmri_1 (-0.170)
   -> tbss_L1_s (-0.169)
   -> tbss_MO_s (-0.165)
   -> tbss_MD (-0.163)
   -> tfmri_c_1 (-0.160)
   -> tbss_L3_s (-0.158)
   -> tbss_L1 (-0.158)
   -> tbss_L2_s (-0.153)
   -> tbss_FA_s (-0.152)
   -> tbss_L3 (-0.141)
   -> tbss_ISOVF (-0.139)
   -> tracts (-0.137)
   -> tbss_MD_s (-0.136)
   -> tbss_ICVF_s (-0.134)
   -> tbss_OD (-0.133)
   -> tbss_L2 (-0.132)
   -> tbss_OD_s (-0.132)
   -> tbss_ICVF (-0.127)
   -> T1_linear (-0.107)
   -> T2_lesions (-0.087)
   -> T1_nonlinear (-0.076)
Var: Head BMD (bone mineral density) (2.0)
   Modalities:
   -> rsfmri_2 (-0.349)
   -> rsfmri_9 (-0.343)
   -> rsfmri_21 (-0.342)
   -> rsfmri_5 (-0.328)
   -> rsfmri_17 (-0.328)
   -> rsfmri_13 (-0.328)
   -> rsfmri_6 (-0.327)
   -> rsfmri_11 (-0.324)
   -> rsfmri_0 (-0.323)
   -> rsfmri_1 (-0.315)
   -> rsfmri_4 (-0.314)
   -> rsfmri_7 (-0.312)
   -> rsfmri_16 (-0.310)
   -> rsfmri_8 (-0

   Modalities:
   -> rsfmri_0 (-0.247)
   -> rsfmri_19 (-0.201)
   -> rsfmri_24 (-0.198)
   -> rsfmri_22 (-0.197)
   -> rsfmri_21 (-0.196)
   -> rsfmri_12 (-0.196)
   -> rsfmri_9 (-0.195)
   -> rsfmri_16 (-0.195)
   -> rsfmri_11 (-0.194)
   -> rsfmri_13 (-0.191)
   -> rsfmri_15 (-0.191)
   -> rsfmri_1 (-0.191)
   -> rsfmri_10 (-0.185)
   -> rsfmri_7 (-0.181)
   -> rsfmri_14 (-0.180)
   -> rsfmri_23 (-0.180)
   -> rsfmri_2 (-0.179)
   -> rsfmri_8 (-0.178)
   -> rsfmri_17 (-0.177)
   -> rsfmri_4 (-0.173)
   -> rsfmri_6 (-0.170)
   -> rsfmri_3 (-0.166)
   -> rsfmri_18 (-0.162)
Var: Heel broadband ultrasound attenuation (right) (2.0)
   Modalities:
   -> rsfmri_0 (-0.212)
   -> rsfmri_21 (-0.192)
   -> rsfmri_5 (-0.185)
   -> rsfmri_24 (-0.181)
   -> rsfmri_7 (-0.178)
   -> rsfmri_4 (-0.177)
   -> rsfmri_11 (-0.177)
   -> rsfmri_17 (-0.174)
   -> rsfmri_9 (-0.173)
   -> rsfmri_13 (-0.172)
   -> rsfmri_8 (-0.172)
   -> rsfmri_15 (-0.171)
   -> rsfmri_20 (-0.170)
   -> rsfmri_2 (-0.170)
   -

   -> tracts (-0.118)
   -> jacobian (-0.117)
   -> rsfmri_9 (-0.117)
   -> rsfmri_12 (-0.117)
   -> tfmri_c_1 (-0.116)
   -> tfmri_5 (-0.116)
   -> tbss_MD (-0.115)
   -> rsfmri_17 (-0.115)
Var: Speed of sound through heel (left) (1.0)
   Modalities:
   -> rsfmri_0 (-0.233)
   -> rsfmri_24 (-0.221)
   -> rsfmri_12 (-0.215)
   -> rsfmri_7 (-0.213)
   -> rsfmri_21 (-0.211)
   -> rsfmri_6 (-0.208)
   -> rsfmri_8 (-0.204)
   -> rsfmri_9 (-0.203)
   -> rsfmri_1 (-0.202)
   -> rsfmri_19 (-0.201)
   -> rsfmri_16 (-0.197)
   -> rsfmri_11 (-0.186)
   -> rsfmri_17 (-0.184)
   -> rsfmri_23 (-0.183)
   -> rsfmri_15 (-0.182)
   -> rsfmri_22 (-0.180)
   -> rsfmri_10 (-0.179)
   -> rsfmri_4 (-0.172)
   -> rsfmri_14 (-0.171)
   -> rsfmri_2 (-0.168)
   -> rsfmri_13 (-0.165)
   -> rsfmri_3 (-0.165)
Var: Speed of sound through heel (left) (2.0)
   Modalities:
   -> rsfmri_0 (-0.206)
   -> rsfmri_24 (-0.182)
   -> rsfmri_21 (-0.182)
   -> rsfmri_7 (-0.174)
   -> rsfmri_5 (-0.173)
   -> rsfmri_8 (-0.173)


   Modalities:
   -> rsfmri_21 (-0.259)
   -> rsfmri_2 (-0.253)
   -> rsfmri_11 (-0.247)
   -> rsfmri_5 (-0.241)
   -> rsfmri_13 (-0.239)
   -> rsfmri_17 (-0.237)
   -> rsfmri_16 (-0.230)
   -> rsfmri_0 (-0.230)
   -> rsfmri_9 (-0.228)
   -> rsfmri_6 (-0.225)
   -> rsfmri_15 (-0.222)
   -> rsfmri_7 (-0.220)
   -> rsfmri_8 (-0.218)
   -> rsfmri_4 (-0.217)
   -> rsfmri_12 (-0.212)
   -> rsfmri_20 (-0.207)
   -> rsfmri_10 (-0.206)
   -> rsfmri_1 (-0.206)
   -> rsfmri_14 (-0.204)
   -> rsfmri_18 (-0.200)
   -> rsfmri_24 (-0.198)
   -> rsfmri_23 (-0.198)
   -> tfmri_5 (-0.197)
   -> rsfmri_3 (-0.196)
   -> rsfmri_22 (-0.190)
   -> rsfmri_19 (-0.183)
   -> tfmri_c_5 (-0.168)
   -> swi (-0.166)
   -> tfmri_2 (-0.165)
   -> jacobian (-0.163)
   -> tfmri_c_2 (-0.159)
   -> tfmri_c_1 (-0.151)
   -> tbss_MO (-0.147)
   -> tbss_FA (-0.143)
   -> tfmri_1 (-0.139)
   -> tbss_ISOVF_s (-0.132)
   -> tbss_MD (-0.130)
   -> tbss_L1 (-0.127)
   -> tbss_L1_s (-0.125)
   -> tbss_OD_s (-0.123)
   -> tbss_OD

In [23]:
variance_threshold = 0.1

for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        
        if prs[0] >= variance_threshold or abs(prs[-1])>= variance_threshold:
            print('Var: {}'.format(name))
            print('   Modalities:')
            for i, mod in enumerate(mods):
                if prs[i] >= variance_threshold or abs(prs[i])>= variance_threshold:
                    print('   -> {} ({:.3f})'.format(mod, prs[i]))
        
    print('\n')

Category: Alcohol
------------------------------------------------
Var: Amount of alcohol drunk on a typical drinking day (0.0)
   Modalities:
   -> tbss_L2 (0.122)
   -> tbss_MD (0.119)
   -> tbss_L3 (0.106)
   -> tbss_L1 (0.105)
   -> tbss_L2_s (0.103)
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> tbss_MD (0.121)
   -> tbss_L2 (0.115)
   -> tbss_L3 (0.103)
   -> tbss_L2_s (0.100)
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> tbss_MD (0.112)
   -> tbss_FA (0.111)
   -> tbss_ISOVF_s (0.111)
   -> tbss_FA_s (0.108)
   -> tbss_L3_s (0.108)
   -> tbss_L2_s (0.107)
   -> tbss_L2 (0.104)
   -> jacobian (0.101)
   -> tbss_L1 (0.101)
   -> tbss_ISOVF (0.101)


Category: Blood Assays
------------------------------------------------


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Diastolic blood pressure, automated reading (0.0)
   Modalities:
   -> T2_lesions (0.105)
   -> tbss_L3_s (0.1

Var: Femur upper neck BMD (bone mineral density) T-score (left) (2.0)
   Modalities:
   -> rsfmri_21 (-0.246)
   -> rsfmri_6 (-0.243)
   -> rsfmri_9 (-0.239)
   -> rsfmri_5 (-0.232)
   -> rsfmri_11 (-0.229)
   -> rsfmri_0 (-0.224)
   -> rsfmri_1 (-0.220)
   -> rsfmri_2 (-0.217)
   -> rsfmri_10 (-0.212)
   -> rsfmri_12 (-0.208)
   -> rsfmri_15 (-0.206)
   -> rsfmri_14 (-0.205)
   -> rsfmri_13 (-0.200)
   -> tfmri_2 (-0.199)
   -> rsfmri_8 (-0.196)
   -> rsfmri_20 (-0.195)
   -> rsfmri_17 (-0.194)
   -> rsfmri_3 (-0.191)
   -> rsfmri_4 (-0.190)
   -> rsfmri_23 (-0.184)
   -> tfmri_5 (-0.176)
   -> rsfmri_16 (-0.173)
   -> rsfmri_7 (-0.172)
   -> rsfmri_19 (-0.171)
   -> rsfmri_18 (-0.170)
   -> rsfmri_22 (-0.170)
   -> rsfmri_24 (-0.160)
   -> tfmri_c_5 (-0.160)
   -> tfmri_1 (-0.159)
   -> tbss_L1 (-0.150)
   -> tfmri_c_2 (-0.149)
Var: Femur upper neck BMD (bone mineral density) T-score (right) (2.0)
   Modalities:
   -> rsfmri_9 (-0.275)
   -> rsfmri_21 (-0.265)
   -> rsfmri_6 (-0.252)

   -> rsfmri_21 (-0.211)
   -> rsfmri_6 (-0.208)
   -> rsfmri_8 (-0.204)
   -> rsfmri_9 (-0.203)
   -> rsfmri_1 (-0.202)
   -> rsfmri_19 (-0.201)
   -> rsfmri_16 (-0.197)
   -> rsfmri_11 (-0.186)
   -> rsfmri_17 (-0.184)
   -> rsfmri_23 (-0.183)
   -> rsfmri_15 (-0.182)
   -> rsfmri_22 (-0.180)
   -> rsfmri_10 (-0.179)
   -> rsfmri_4 (-0.172)
   -> rsfmri_14 (-0.171)
   -> rsfmri_2 (-0.168)
   -> rsfmri_13 (-0.165)
   -> rsfmri_3 (-0.165)
Var: Speed of sound through heel (right) (0.0)
   Modalities:
   -> rsfmri_0 (-0.168)
   -> rsfmri_24 (-0.158)
   -> rsfmri_8 (-0.157)
   -> rsfmri_15 (-0.156)
   -> rsfmri_1 (-0.153)
   -> rsfmri_4 (-0.147)
   -> rsfmri_5 (-0.143)
   -> rsfmri_7 (-0.141)
   -> rsfmri_6 (-0.131)
   -> rsfmri_19 (-0.131)
   -> rsfmri_11 (-0.130)
   -> rsfmri_21 (-0.130)
   -> rsfmri_14 (-0.129)
   -> rsfmri_2 (-0.128)
   -> rsfmri_16 (-0.125)
   -> rsfmri_18 (-0.124)
   -> rsfmri_10 (-0.121)
   -> rsfmri_12 (-0.119)
   -> tbss_MD (-0.117)
   -> vbm (-0.116)
   -> rsfmr

In [20]:
mods

('tbss_OD', 'tbss_L2_s', 'tbss_L1_s', 'tbss_MD_s')

In [21]:
counter = 0
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()[0]
        if 'T1_nonlinear' in mods or 'T1_linear' in mods or 'T2_nonlinear' in mods:
            pass
        else:
            print('Var: {}'.format(name))
            counter += 1
    print('\n')
    
print("There are {} vars independent from the conventional modalities".format(counter))

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
Var: Alcohol intake frequency. (2.0)
Var: Average weekly red wine intake (0.0)
Var: Average weekly spirits intake (2.0)
Var: Frequency of consuming six or more units of alcohol (0.0)
Var: Frequency of drinking alcohol (0.0)


Category: Blood Assays
------------------------------------------------
Var: C-reactive protein (0.0)
Var: Mean corpuscular haemoglobin (0.0)
Var: Mean corpuscular volume (0.0)


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Cardiac index during PWA (2.0)
Var: Cardiac index during PWA (2.1)
Var: Cardiac output during PWA (2.0)
Var: Central pulse pressure during PWA (2.0)
Var: Central systolic blood pressure during PWA (2.0)
Var: Diastolic blood pressure, automated reading (2.0)
Var: Diastolic blood pressure, automated reading (2.1)
Var: Diastolic brachial blood pressure (2.0)
Var: End systolic pressure during

In [22]:
counter = 0
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()[0]
        if 'T1_nonlinear' in mods or 'T1_linear' in mods or 'T2_nonlinear' in mods:
            print('Var: {}'.format(name))
            counter += 1
        else:
            pass
    print('\n')
    
print("There are {} vars in the conventional modalities".format(counter))

Category: Alcohol
------------------------------------------------
Var: Amount of alcohol drunk on a typical drinking day (0.0)


Category: Blood Assays
------------------------------------------------


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Diastolic blood pressure, automated reading (0.0)
Var: Diastolic blood pressure, automated reading (0.1)
Var: Systolic blood pressure, automated reading (0.0)
Var: Systolic blood pressure, automated reading (0.1)


Category: Cognitive Tests
------------------------------------------------


Category: Diet
------------------------------------------------


Category: Medical History
------------------------------------------------
Var: Diabetes diagnosed by doctor (0.0)
Var: Diabetes diagnosed by doctor (2.0)
Var: Diagnoses - ICD10 (I10 - I10 Essential (primary) hypertension)
Var: Diagnoses - secondary ICD10 (E109 - E10.9 Without complications)
Var: Diagnoses - secondary ICD10 (I10 - I10 E